In [9]:
import openai
import numpy as np
import json
import os

OPENAI_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_KEY

In [10]:
# with open("objects.txt") as f:
#     objects = [line.rstrip() for line in f]

# with open("actions.txt") as f:
#     actions = [line.rstrip() for line in f]

# 1. given the task, figure out what kinds of objects we might need with LLM
# 2. see if we have every object in sight (or see how similar our objects are)
# 3. tell LLM our limitations (if any)
# 4. ask for instructions given the exact objects we have 

In [104]:
# given a task, returns the objects and their imporances needed for a specific task
def get_objects_needed(task):
    chat_lm_id = "gpt-3.5-turbo"
    # translation_lm_id = "text-embedding-ada-002"
    system_message = """
    You are a large language model tasked with planning a robot's actions.
    """
    def command(sample_task):
        return f"""
        list all objects i might need to interact with to perform the task '{sample_task}'? 
        write each object in as few words as possible with no ambiguity. list each object in as few words as possible. 
        next to each object give a score from 1 to 10 scoring its importance for the given task in the format [<object>:<score>,<object>:<score>, etc.]
        """
    original_messages=[
    {"role": "system", "content": system_message},
        {"role": "user", "content": command("take out the trash")},
        {"role": "assistant", "content": """trash can:10,trash bag:6,dumpster:8,"""},
        {"role": "user", "content": command("do the dishes")},
        {"role": "assistant", "content": """dishes:10,sink:10,soap:9,sponge:5,drying rack:2"""},
        {"role": "user", "content": command(task)},
    ]
    res = openai.ChatCompletion.create(
        model=chat_lm_id,
        messages=original_messages,
        temperature=1.5
    )
    # print(res)
    objects = res["choices"][0]["message"]["content"]
    objs_with_importance = []
    for obj_with_score in objects.split(","):
        splitted = obj_with_score.split(":")
        try:
            objs_with_importance.append((splitted[0].lstrip(" "), int(splitted[1])))
        except:
            raise ValueError(f"improper format: {obj_with_score}")
        
    return objs_with_importance

In [105]:
get_objects_needed("turn on the lights")

[('power switch', 10), ('light bulb', 8), ('lamp', 6), ('light switch', 10)]

In [91]:
chat_lm_id = "gpt-3.5-turbo"
translation_lm_id = "text-embedding-ada-002"
system_message="""
You are a large language model for planning a robot's movements to achieve a given task. 
The user will provide you with a task and you will instruct the user (a robot) to perform the task.
The robot may move to an object, pick it up, or place it in a new location.
The robot may also move to a location, and place an object there.
Give a command in the form of "move to <object> and pick up <object> and place it in <object>".
Objects: trash can, cup, table, trash, dumpster, table
"""
original_messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "Take out the trash."},
    {"role": "assistant", "content": """
    1. Move to the trash can
    2. Pick up the trash
    3. Move to the dumpster
    4. Place the trash in the dumpster
    """},
    {"role": "user", "content": "You have 10 cups. Arrange them to form a circle. give step by step instructions for each cup"},
]

In [3]:
res = openai.ChatCompletion.create(
    model=chat_lm_id,
    messages=original_messages,
    temperature=1.5
)

In [4]:
res_message = res["choices"][0]["message"]
res_message

<OpenAIObject at 0x7fde29646700> JSON: {
  "content": "Sure, I'd be happy to help! Since all ten cups must be included to form a circle, you can arrange them in a circular arrangement in which the cups are placed equidistantly around a perfect circle.\n\nHere are the step by step instructions:\n\n1. Pick up cup #1 and place it in front of you at the center of the circle\n2. Pick up cup #2 and place it to the left of cup #1.\n3. Pick up cup #3 and place it to the left of cup #2.\n4. Pick up cup #4, move past cups #3 and  draw left arm  fully extended without colliding with temporary growing formation without damaging pre-determined co-planarity Law then reach into middle of these opened-friendly-space to put in final placement and fix the end that the began has string source associated initial corner center placement.\n5. Do the same extending your leg diagonal opposite tray he ever cup around ensures every cup if supported throughout this adjustment moves avoid destruct support last po

In [11]:
original_messages.append(res_message.to_dict())
original_messages.append({
    "role": "user",
    "content": "Where can I find the trash can. I am currently in the bathroom."
})

In [6]:
res2 = openai.ChatCompletion.create(
    model=chat_lm_id,
    messages=original_messages
)

In [7]:
res2

<OpenAIObject chat.completion id=chatcmpl-71choifxgj1Hxhqy3DXmjeRNlZRhp at 0x7fa1ee388130> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Typically, a trash can is placed in different locations depending on the house or building. In most cases, the trash can is usually found in rooms such as the kitchen, living room or just about any common area in the house. However, some houses may also have small bins in each room, including the bathroom. \nIf you are currently in the bathroom and you are looking for the trash can, I would suggest you look around for any bin or container in the bathroom. You may also consider asking someone you live with or check the house layout to find out where the trash cans are located in your house.",
        "role": "assistant"
      }
    }
  ],
  "created": 1680621468,
  "id": "chatcmpl-71choifxgj1Hxhqy3DXmjeRNlZRhp",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usag

In [38]:
om.update("hello")

ValueError: dictionary update sequence element #0 has length 1; 2 is required